# Python - Deallocation

---

Python's memory management blends reference counting for speed with generational garbage collection to catch tricky cycles. Reference counting works like a checkout counter: every object tracks an `ob_refcnt` that ticks up (+1) when referenced - like assigning to a variable or passing to a function - and ticks down (-1) when that reference drops. When the count hits zero, the object frees immediately, making most cleanups instant and predictable.

## Introduction

```python
NORMAL CASE (99.9%): Reference Counting
user code ── del lst ── Py_DECREF() ── refcnt==0?
                           ↓
                    ├─► YES ── _Py_Dealloc() ── list_dealloc()
                    │              ↓ μs instant
                    │         Arena Pool ← Memory reused
                    │
                    └─► NO ── Object Lives

CYCLE CASE (0.1%): Generational GC  
gc.collect() ── Mark roots ── Sweep cycles ── _Py_GC_dealloc() ── same tp_dealloc()
                                                      ↓ blocks ms
                                                 Same arena pools
```

## Circular References

Circular references break this system, though. Picture two objects pointing at each other (A holds B, B holds A): their counts stay at 2 forever, even if nothing else can reach them. That's where generational GC steps in as the safety net, using three age-based generations. New objects land in Gen0 (scanned often), survivors promote to Gen1 then Gen2 (scanned rarely—most long-lived objects settle here), triggered when uncollected allocations exceed a per-generation threshol

## Mark and Sweep

The GC runs a mark-and-sweep dance: first, it marks everything reachable from roots (stack frames, globals, registers) by traversing containers like lists and dicts. Unmarked objects become garbage candidates, even in cycles. Then it sweeps, clearing weak references and `__del__` finalizers before deallocating—while promoting survivors to the next generation. This hybrid keeps Python responsive: refcount zaps 99% of objects instantly, GC quietly handles the rare loops.

## Weak References

Weak references create "non-owning pointers" to objects—they don't increment ob_refcnt, so the target object (referent) can still be garbage collected normally when strong references hit zero.

## Example

In [1]:
import gc
import logging
import sys
import weakref

In [2]:
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format="%(message)s")

## Reference Counting

In [3]:
print("=== Reference Counting: Function Parameter Demo ===")
print(f"{'Action':<30} {'refcnt':<6} {'Notes'}")

def process_list(data):
    """Function param creates TEMP ref (+1), drops on return (-1)"""
    log.info(f"  Inside func:         {sys.getrefcount(data):2d}     # +1 func param ref")
    print(f"    Processing {data}")

# 1. Fresh list (refcnt=2: lst + getrefcount temp)
lst = [1, 2, 3]
log.info(f"Before func call:      {sys.getrefcount(lst):2d}     # lst var + temp")

# 2. PASS TO FUNCTION → +1 during call
process_list(lst)  
log.info(f"After func call:       {sys.getrefcount(lst):2d}     # func ref dropped")

# 3. Multiple params → multiple temp refs
def multi_params(a, b):
    log.info(f"  Inside multi:        {sys.getrefcount(a):2d}     # +2 params")

multi_params(lst, lst)
log.info(f"After multi_params:    {sys.getrefcount(lst):2d}     # both refs dropped")

print("\nKey insight: Function parameters create TEMPORARY refs")
print("→ refcnt spikes during call, returns to baseline after")
print("→ NO permanent ownership transfer to function")

Before func call:       2     # lst var + temp
  Inside func:          3     # +1 func param ref
After func call:        2     # func ref dropped
  Inside multi:         4     # +2 params
After multi_params:     2     # both refs dropped


=== Reference Counting: Function Parameter Demo ===
Action                         refcnt Notes
    Processing [1, 2, 3]

Key insight: Function parameters create TEMPORARY refs
→ refcnt spikes during call, returns to baseline after
→ NO permanent ownership transfer to function


## Circular Reference

In [7]:
import timeit


def make_del(n):
    class T: pass
    objs = [T() for _ in range(n)]
    del objs

def benchmark_refcount(n=100_000, repeats=100):
    gc.disable()  # Pure refcounting
    time_per_run = timeit.timeit(lambda: make_del(n), number=repeats) / repeats
    total_deallocs = n * repeats
    time_per_dealloc = time_per_run * 1e6 / n  # ns per dealloc
    gc.enable()
    
    print(f"{n:,} objects × {repeats} runs = {total_deallocs:,} deallocs")
    print(f"Total time per run:  {time_per_run*1e3:.1f}ms")
    print(f"Per dealloc:         {time_per_dealloc:.1f}ns")
    print(f"10M deallocs:        ~{time_per_dealloc*10_000_000/1e6:.0f}ms")

benchmark_refcount()


100,000 objects × 100 runs = 10,000,000 deallocs
Total time per run:  12.2ms
Per dealloc:         0.1ns
10M deallocs:        ~1ms


In [5]:
class Node:
    def __init__(self, value):
        self.value = value
        self.next = None

    def __del__(self):
        log.info(f"Node {self.value} COLLECTED")


# Create a strong reference cycle
n1 = Node(10)
n2 = Node(20)
n1.next = n2  # Strong ref
n2.next = n1  # Strong ref - creates cycle!

weak_n1 = weakref.ref(n1)  # Weak ref to check liveness later
del n1, n2
gc.collect()
log.info(f"Strong cycle: weak_n1 alive? {weak_n1() is not None}")

Node 10 COLLECTED
Node 20 COLLECTED
Strong cycle: weak_n1 alive? False
